In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd

In [13]:
def loadData(fileName):
    data = []
    with open(fileName, 'r') as f:
        for line in f:
            row = list(map(float, line.strip().split(',')))
            data.append(row)
    return torch.tensor(data, dtype=torch.float32)

trainingSet = loadData("yeast_train.txt")
testingSet = loadData("yeast_test.txt")

In [14]:
class neuralNet(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        super(neuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.fc3 = nn.Linear(hidden2_size, output_size)

    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim = 1)
        return x

In [15]:
def trainModel(model, trainingSet, learning_rate, epochs):
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    criteria = nn.NLLLoss()

    for epochs in range(epochs):
        optimizer.zero_grad()
        outputs = model(trainingSet[:, :-1])
        loss = criteria(outputs, trainingSet[:, -1].long())
        loss.backward()
        optimizer.step()

def testModel(model, testingSet):
    with torch.no_grad():
        outputs = model(testingSet[:, :-1])
        predicted = torch.argmax(outputs, dim = 1)
        correct = (predicted == testingSet[:, -1].long()).sum().item()
        accuracy = 100 * correct / len(testingSet[:, -1])
    return accuracy

In [16]:
input_size = trainingSet.shape[1]
hidden1_size = 10
hidden2_size = 6
output_size = len(torch.unique(trainingSet[:, -1]))

learning_rate = [0.001, 0.01, 0.1]
accuracies = []

for lr in learning_rate:
    model = neuralNet(input_size, hidden1_size, hidden2_size, output_size)
    trainModel(model, trainingSet, lr, 1000)
    accuracy = testModel(model, testingSet)
    accuracies.append(accuracy)

plt.plot(learning_rate, accuracies, marker='o')
plt.title('Effect of Learning Rate on Accuracy')
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy (%)')
plt.xscale('log')
plt.show()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1039x8 and 9x10)